In [1]:
import os
os.chdir('../')

## Imports

In [2]:
import pandas as pd
from src.data_preprocess import DataProcessor

## Data Collect

In [3]:
df_imobs  = pd.read_csv("notebooks/outputs/df_imobs.csv").rename(columns={"dt_ativacao": "dt_calendar"})
df_imobs["dt_calendar"] = pd.to_datetime(df_imobs["dt_calendar"])

print(df_imobs.shape)
df_imobs.head()

(92656, 20)


,dt_calendar,id_imobiliaria,lat_imob,long_imob,is_active,is_activated,is_commun_first_90days,is_debelado,is_debelado_first_4months,is_indemnified,indemnity_value,is_indemn_first_6months,vl_locacao,is_churn,exonerated_first_6months,rating_A,rating_B,rating_C,rating_D,rating_E
0,2016-04-01,30,-26.26,-48.85,0.0,2,0,0,0,0,0.0,0,945.00,1,0,0.0,0.0,0.0,0.0,0.0
1,2016-06-01,30,-26.26,-48.85,1.0,2,0,0,0,0,0.0,0,778.30,0,0,0.0,0.0,0.0,0.0,0.0
2,2016-07-01,81,-27.60,-48.61,0.0,1,0,0,0,0,0.0,0,1200.00,0,0,0.0,0.0,0.0,0.0,0.0
3,2016-07-01,95,-26.89,-49.08,0.0,1,0,0,0,0,0.0,0,1000.00,0,0,0.0,0.0,0.0,0.0,0.0
4,2016-08-01,95,-26.89,-49.08,1.0,3,0,0,0,0,0.0,0,1533.33,0,0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Novas imobiliárias
(
    df_imobs["is_active"].apply(lambda x: (x < 100))
    .to_frame("new_imob")
    .groupby("new_imob").size()
    .to_frame("count")
)

,count
new_imob,
False,10465
True,82191


In [5]:
df_imobs.groupby("id_imobiliaria", as_index=False)["is_active"].mean()

,id_imobiliaria,is_active
0,25,3.571429
1,30,33.181818
2,35,6.875000
3,36,0.500000
4,38,151.065574
...,...,...
8194,30484,0.000000
8195,30501,0.000000
8196,30537,0.000000
8197,30547,0.000000


In [6]:
df = df_imobs.loc[df_imobs["is_active"] > 100, :].reset_index(drop=True)

In [7]:
# 3% de target positivo
df["target"] = df["is_indemn_first_6months"].transform(
    lambda x: 1 if x >= 1 else 0 
)

df.groupby("target").size()

target
0    10015
1      349
dtype: int64

## Shift das Features

In [8]:
df_imobs.columns

Index(['dt_calendar', 'id_imobiliaria', 'lat_imob', 'long_imob', 'is_active',
       'is_activated', 'is_commun_first_90days', 'is_debelado',
       'is_debelado_first_4months', 'is_indemnified', 'indemnity_value',
       'is_indemn_first_6months', 'vl_locacao', 'is_churn',
       'exonerated_first_6months', 'rating_A', 'rating_B', 'rating_C',
       'rating_D', 'rating_E'],
      dtype='object')

In [9]:
processor = DataProcessor(df)

In [10]:
# Correção do deslocamento de +1 mês da variável is_active
df_shift = processor.features_shift(
    group_by="id_imobiliaria",
    columns_to_shift={
        -1: ["is_active"]

    }
)

In [11]:
df_model = df_shift[~df_shift.is_active.isnull()]

In [12]:
df_model.dt_calendar.describe()

/var/folders/nc/c031_2vx4zgf4ldqyn14mvvm0000gq/T/ipykernel_15129/895287199.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_model.dt_calendar.describe()


count                    9821
unique                     68
top       2022-12-01 00:00:00
freq                      445
first     2017-05-01 00:00:00
last      2022-12-01 00:00:00
Name: dt_calendar, dtype: object

In [13]:
df_model.to_csv("notebooks/outputs/df_model.csv", index=False)